# Add Dependences

In [1]:
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/run_event.jl")

@make_struct_func Cubic

force_func (generic function with 1 method)

# Add Data

In [2]:
parameters = Model(
    Cubic(
        μ₁       = 0.1,
        rₘᵢₙ     = 2,
        rₘₐₓ     = 3
    ),
    Contractile(
        fₚ       = 0.0000001
    ),
    Time(
        t_f      = 100000, 
        dt       = 0.1
    ),
    Neighbor(
        n_knn    = 50,
        nn       = 12
    ),
    Geometry(
        R_agg    = 9,
        position = [
            0.0    0.0    0.0
        ]
    ),
    Simulation(
        n_text   = 200,
        path     = "BBB.xyz"
    )
)

Model(Cubic(0.1, 2.0, 3.0), Contractile(1.0e-7), Time(100000.0, 0.1), Neighbor(50, 12), Geometry(9.0, [0.0 0.0 0.0]), Simulation(200, "BBB.xyz"))

In [3]:
println("Loading Data")
CUDA.@time X = Float32.(readdlm("../../data/init/Sphere/$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu
println("\n Generating Struct")
CUDA.@time Agg = Aggregate(Neighbor(50, 12), PositionCell(X));

## Var before func
println("\n Loading Force Function")
CUDA.@time force(r) =  force_func(parameters.Force::Cubic, r)

## Adding Sum of Aggregates
println("\n Adding Aggregates from Position")
CUDA.@time SumAgg(Agg::Aggregate, parameters::Model)

# Simulating
println("\n Simulation")
CUDA.@time simulate(true, parameters, Cubic(1.0, 2.0, 3.0), Agg)

Loading Data
  1.543347 seconds (2.28 M CPU allocations: 132.504 MiB, 2.90% gc time) (1 GPU allocation: 6.305 KiB, 3.24% memmgmt time)

 Generating Struct
  2.276591 seconds (2.30 M CPU allocations: 134.787 MiB, 1.95% gc time) (12 GPU allocations: 4.813 MiB, 0.02% memmgmt time)

 Loading Force Function
  0.000053 seconds (24 CPU allocations: 1.634 KiB)

 Adding Aggregates from Position
 34.833225 seconds (50.87 M CPU allocations: 2.869 GiB, 4.64% gc time) (20 GPU allocations: 4.850 MiB, 0.00% memmgmt time)

 Simulation


Progress:  13%|███▎                     |  ETA: 0:24:47m21m

LoadError: InterruptException: